In [1]:
## This Jupyter notebook reproduces the results for WDL in Table 1

In [2]:
# import packages
import os, sys
sys.path.append('../../../lib/')
import WDL as wp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

In [3]:
# load data
id_setting = 4 # \omega = [0.1, 0.2, 0.5, 1, 2]
X = pd.read_csv('../../../data/simulation/setting_' + str(id_setting) + '/dat_X.csv').to_numpy()
Y = pd.read_csv('../../../data/simulation/setting_' + str(id_setting) + '/dat_Y.csv').to_numpy()
loc_cv = pd.read_csv('../../../data/simulation/setting_' + str(id_setting) + '/dat_CV.csv').to_numpy().flatten()

In [4]:
# nested cross validation
n_dist = Y.shape[0]
n_levs = 100
n_fold = np.max(loc_cv) + 1
q_vec = np.arange(1, n_levs) / n_levs
## transform Y
Q_mat = np.array([np.quantile(Y[i], q_vec) for i in range(n_dist)])
Q_test = np.zeros(Q_mat.shape)

K_list = [2, 3, 5]
lr_list = [1e-1, 1e-2]
n_iter = 1000
## outer loop
time_start = datetime.now()
print('Start training:', time_start)
for i in range(n_fold):
    print('This is fold', str(i+1))
    X_train = X[loc_cv != i]
    Y_train = Q_mat[loc_cv != i]
    X_test = X[loc_cv == i]
    Y_test = Q_mat[loc_cv == i]
    ## inner parameter selection
    X_t_in, X_v_in, Y_t_in, Y_v_in = train_test_split(X_train, Y_train, test_size=0.25, random_state=2022)
    par_combo = [(K, lr) for K in K_list for lr in lr_list]
    loss_ = []
    iters_ = []
    for K_mix, lr in par_combo:
        print(K_mix, lr)
        res_init = wp.WDL(X_t_in, Y_t_in, X_v_in, Y_v_in,
                          q_vec=q_vec, K=K_mix, max_iter=n_iter, warm_up=1, max_depth=1, 
                          patience=10, lr=lr, random_state=2022)
        iters_.append(res_init['iter_best'])
        loss_.append(res_init['val_loss'][res_init['iter_best']])
    ## choose the best params
    K_best, lr_best = par_combo[np.argmin(np.array(loss_))]
    iter_best = iters_[np.argmin(np.array(loss_))]
    print('Loss:', loss_)
    print('Best:', K_best, lr_best, iter_best)
    ## retrain the model over the training set
    res = wp.WDL(X_train, Y_train, X_test, Y_test, q_vec=q_vec, 
                 K=K_best, max_iter=iter_best, warm_up=1, 
                 max_depth=1, early_stop=False, lr=lr_best, random_state=2022)
    
    
    alpha_test = np.zeros((X_test.shape[0], K_best))
    mu_test = np.zeros((X_test.shape[0], K_best))
    sigma_test = np.zeros((X_test.shape[0], K_best))
    
    v_lr = np.array([1] + [lr_best] * iter_best)
    for k in range(K_best):
        alpha_test[:, k] = wp.pred_boost(X_test, res['alpha'][k], lr_=v_lr, n_term=iter_best)
        mu_test[:, k] = wp.pred_boost(X_test, res['mu'][k], lr_=v_lr, n_term=iter_best)
        sigma_test[:, k] = np.exp(wp.pred_boost(X_test, res['sigma'][k], lr_=v_lr, n_term=iter_best))
    
    n_test = Y_test.shape[0]
    pi_test = np.exp(alpha_test)
    pi_test = (pi_test.T / np.sum(pi_test, axis=1)).T
    Q_test[loc_cv == i] = [wp.qgmm1d(q_vec, mu_test[j], sigma_test[j], pi_test[j]) for j in range(n_test)]
print('Done!')
print('Time:', datetime.now() - time_start )

Start training: 2021-09-11 22:26:47.284286
This is fold 1
2 0.1
2 0.01
3 0.1
3 0.01
5 0.1
5 0.01
Loss: [1.0066234555486564, 1.035182994457966, 1.0730263388070131, 1.1108563536921738, 1.0933097274882972, 1.1089402699187894]
Best: 2 0.1 40
This is fold 2
2 0.1
2 0.01
3 0.1
3 0.01
5 0.1
5 0.01
Loss: [1.0732741087445732, 1.0836639057172175, 1.0662676340875912, 1.0885668174537841, 1.1510590426844258, 1.1738985578203667]
Best: 3 0.1 207
This is fold 3
2 0.1
2 0.01
3 0.1
3 0.01
5 0.1
5 0.01
Loss: [1.1625455374468388, 1.1672190355084013, 1.1930910612263275, 1.2179618632283293, 1.2704952776874063, 1.2873273525306594]
Best: 2 0.1 27
This is fold 4
2 0.1
2 0.01
3 0.1
3 0.01
5 0.1
5 0.01
Loss: [1.1922378908521385, 1.2045553692208755, 1.2546910389229198, 1.2888441234392123, 1.2809532303674678, 1.33052184907812]
Best: 2 0.1 32
This is fold 5
2 0.1
2 0.01
3 0.1
3 0.01
5 0.1
5 0.01
Loss: [0.8394160678621866, 0.8645438915561346, 0.8590763501918894, 0.8967114590667549, 0.9120452990083543, 0.934406005657

In [5]:
# evaluate the results
RMSE = np.mean((Q_mat - Q_test)**2)
var_y = np.mean((Q_mat - np.mean(Q_mat, axis=0))**2)
R_sq = 1 - RMSE / var_y
print('Test loss:', RMSE)
print('Test R-squared:', R_sq)

Test loss: 1.0867950524265513
Test R-squared: 0.3402540477459597
